#**Time Series Analysis**

1. Analyze the trend of deal creation over time and its relationship with calls.

2. Examine the distribution of deal closing times and the duration of the period from creation to closing.

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None) #display all columns (so that they are not hidden with «…»)

In [ ]:
deals = pd.read_pickle("deals_df.pkl")
calls = pd.read_pickle("calls_df.pkl")

In [ ]:
deals.head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,_open_deal,Payment Category,Offer Category,_SLA_hours
0,5805028000056864695,Ben Hall,NaT,Unknown,New Lead,Unknown,03.07.23women,0 days 00:00:00,v16,women,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 15:30:00,0,0,NaN,NaN,5805028000056849495,Unknown,Unknown,True,Unknown,Unknown,0.000000
1,5805028000056859489,Ulysses Adams,NaT,Unknown,New Lead,Unknown,Unknown,0 days 00:00:00,Unknown,Unknown,Organic,Unknown,Web Developer,Morning,2024-06-21 15:23:00,6,0,0.0,2000.0,5805028000056834471,Unknown,Unknown,True,No Payment,Regular Offer,0.000000
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_AT,0 days 00:26:43,b1-at,21_06_2024,Telegram posts,Unknown,Unknown,Unknown,2024-06-21 14:45:00,0,0,NaN,NaN,5805028000056854421,Unknown,Unknown,False,Unknown,Unknown,0.445278
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_DE,0 days 01:00:03.999999999,bloggersvideo14com,recentlymoved,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 13:32:00,0,0,NaN,NaN,5805028000056889351,Unknown,Unknown,False,Unknown,Unknown,1.001111
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_DE,0 days 00:53:12.000000001,website,Unknown,Google Ads,Unknown,Unknown,Unknown,2024-06-21 13:21:00,0,0,NaN,NaN,5805028000056876176,Unknown,Unknown,False,Unknown,Unknown,0.886667


In [ ]:
calls.head()

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM,_missing_contact
0,5805028000000805001,2023-06-30 08:43:00,John Doe,NaN,Inbound,171,Received,Unknown,Unknown,True
1,5805028000000768006,2023-06-30 08:46:00,John Doe,NaN,Outbound,28,Attended Dialled,Completed,NO,True
2,5805028000000764027,2023-06-30 08:59:00,John Doe,NaN,Outbound,24,Attended Dialled,Completed,NO,True
3,5805028000000787003,2023-06-30 09:20:00,John Doe,5805028000000645014,Outbound,6,Attended Dialled,Completed,NO,False
4,5805028000000768019,2023-06-30 09:30:00,John Doe,5805028000000645014,Outbound,11,Attended Dialled,Completed,NO,False


### **1 — Trend of Deal Creation vs Calls**

In [ ]:
# 1. Aggregation of deals and calls by month
deals_monthly = deals.groupby(deals['Created Time'].dt.to_period('M')).size()
calls_monthly = calls.groupby(calls['Call Start Time'].dt.to_period('M')).size()

In [ ]:
# Only completed calls (Call Duration > 0)
calls_done_monthly = (calls[calls['Call Duration (in seconds)'] > 0].groupby(calls['Call Start Time'].dt.to_period('M')).size())
calls_done_monthly

,0
Call Start Time,
2023-06,7
2023-07,1414
2023-08,2996
2023-09,3717
2023-10,5399
2023-11,5690
2023-12,5836
2024-01,8120
2024-02,7846


In [ ]:
# Convert indices back to datetime
deals_monthly.index = deals_monthly.index.to_timestamp()
calls_monthly.index = calls_monthly.index.to_timestamp()
calls_done_monthly.index = calls_done_monthly.index.to_timestamp()

In [ ]:
# 2. Combine into a single DataFrame for convenience
trend_df = pd.DataFrame({
    "Deals": deals_monthly,
    "Calls (all)": calls_monthly,
    "Calls (done)": calls_done_monthly
}).fillna(0)

In [ ]:
trend_df

,Deals,Calls (all),Calls (done)
2023-06-01,0.0,7,7
2023-07-01,654.0,1935,1414
2023-08-01,1085.0,4251,2996
2023-09-01,1079.0,5156,3717
2023-10-01,1598.0,7102,5399
2023-11-01,1947.0,7170,5690
2023-12-01,1811.0,7099,5836
2024-01-01,2243.0,9804,8120
2024-02-01,2184.0,9599,7846
2024-03-01,2252.0,10081,8001


In [ ]:
# 3. Line chart of dynamics

palette = {
    "gold":   "#FFDFA6",   # светлое золото
    "orange": "#F5A623",   # оранжевый акцент
    "dark":   "#B86B00",   # тёмно-оранжевый
    "grey":   "#4A4A4A",   # тёмно-серая линия для Deals
    "line":   "#D9D9D9",   # тонкие линии/оси
    "char":   "#2E2E2E",   # текст
    "bg":     "rgba(0,0,0,0)"  # прозрачный фон
}

fig = px.line(
    trend_df,
    x=trend_df.index,
    y=["Deals", "Calls (all)", "Calls (done)"],
    markers=True,
    title="Trend of Deal Creation and Calls Over Time")

fig.update_traces(
    line=dict(width=2),
    marker=dict(size=8, line=dict(color=palette["line"], width=1)))

fig.data[0].line.color = palette["grey"]
fig.data[1].line.color = palette["gold"]
fig.data[2].line.color = palette["orange"]

fig.update_traces(textposition="top center", selector=dict(mode="markers+lines"))

fig.update_layout(
    title=dict(
        text="Trend of Deal Creation and Calls Over Time",
        x=0.0,
        font=dict(size=18, color=palette["char"])),
    xaxis_title="Month",
    yaxis_title="Number",
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=60, r=40, t=60, b=60),
    xaxis=dict(showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    legend=dict(
        orientation="h", yanchor="bottom", y=1.02,
        xanchor="left", x=0, bgcolor="rgba(0,0,0,0)"))

fig.show()


### Наблюдения
- **Звонки (Calls all):**
  - Рост с ~2 000 в июле 2023 до пика ~13 300 в апреле 2024.
  - Снижение до ~8 500 в июне 2024.
- **Завершённые звонки (Calls done):**
  - Динамика почти совпадает с общими звонками.
  - Пик в марте–апреле 2024 (~10 000), затем падение до ~6 200 в июне 2024.
- **Сделки (Deals):**
  - Рост с ~650 в июле 2023 до пика ~3 080 в апреле 2024.
  - В июне 2024 снижение до ~1 670.

In [ ]:
# 4. Conversion (Deals / Calls)

palette = {
    "grey":   "#4A4A4A",
    "line":   "#D9D9D9",
    "char":   "#2E2E2E",
    "bg":     "rgba(0,0,0,0)"
}

# Conversion Rate Over Time
trend_df["Conversion Rate"] = trend_df["Deals"] / trend_df["Calls (all)"]

fig2 = px.line(
    trend_df,
    x=trend_df.index,
    y="Conversion Rate",
    markers=True,
    title="Conversion Rate Over Time (Deals / Calls)")

fig2.update_traces(
    line=dict(color=palette["grey"], width=2),
    marker=dict(size=8, line=dict(color=palette["line"], width=1)),
    text=trend_df["Conversion Rate"].round(2),
    textposition="top center")

fig2.update_layout(
    title=dict(
        text="Conversion Rate Over Time (Deals / Calls)",
        x=0.0,
        font=dict(size=18, color=palette["char"])),
    xaxis_title="Month",
    yaxis_title="Conversion Rate",
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=60, r=40, t=60, b=60),
    xaxis=dict(showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    legend=dict(visible=False))

fig2.show()


In [ ]:
palette = {
    "gold":   "#D4A017",   # тёмно-золотой
    "orange": "#E67E22",   # насыщенный оранжевый
    "grey":   "#2E2E2E",   # насыщенный тёмно-серый для Deals
    "line":   "#B0B0B0",   # линии осей
    "char":   "#1A1A1A",   # текст
    "bg":     "rgba(0,0,0,0)",  # прозрачный фон
    "green":  "#27AE60"    # насыщенный зелёный для Conversion
}

trend_df["Conversion Rate"] = trend_df["Deals"] / trend_df["Calls (all)"]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=trend_df.index,
    y=trend_df["Conversion Rate"] * 100,
    name="Conversion Rate",
    marker_color=palette["green"],
    yaxis="y2",
    text=trend_df["Conversion Rate"].round(2),
    textposition="outside",
    opacity=0.3))

fig.add_trace(go.Scatter(
    x=trend_df.index, y=trend_df["Deals"],
    mode="lines+markers", name="Deals",
    line=dict(color=palette["grey"], width=3),
    marker=dict(size=9, color=palette["grey"],
                line=dict(color=palette["line"], width=1))))

fig.add_trace(go.Scatter(
    x=trend_df.index, y=trend_df["Calls (all)"],
    mode="lines+markers", name="Calls (all)",
    line=dict(color=palette["gold"], width=3),
    marker=dict(size=9, color=palette["gold"],
                line=dict(color=palette["line"], width=1))))

fig.add_trace(go.Scatter(
    x=trend_df.index, y=trend_df["Calls (done)"],
    mode="lines+markers", name="Calls (done)",
    line=dict(color=palette["orange"], width=3),
    marker=dict(size=9, color=palette["orange"],
                line=dict(color=palette["line"], width=1))))

fig.update_layout(
    title=dict(
        text="Trend of Deals, Calls and Conversion Rate Over Time",
        x=0.0,
        font=dict(size=18, color=palette["char"])),
    xaxis=dict(
        title="Month",
        showgrid=False,
        linecolor=palette["line"],
        tickcolor=palette["line"]),
    yaxis=dict(
        title="Number (Deals & Calls)",
        showgrid=False,
        linecolor=palette["line"],
        tickcolor=palette["line"]),
    yaxis2=dict(
        title="Conversion Rate (%)",
        overlaying="y",
        side="right",
        showgrid=False,
        linecolor=palette["line"],
        tickcolor=palette["line"]),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=60, r=60, t=60, b=60),
    legend=dict(
        orientation="h", yanchor="bottom", y=1.02,
        xanchor="left", x=0, bgcolor="rgba(0,0,0,0)"))

fig.show()





- **Конверсия (Deals / Calls):**
  - Июль 2023 — **0.34** (34 сделки на 100 звонков).
  - Август–декабрь 2023 — постепенное падение (0.20–0.26).
  - Май–июнь 2024 — минимум **0.17–0.19**.

### Выводы
1. Количество звонков и сделок активно росло до весны 2024 года, после чего пошёл спад.  
2. Пики сделок и звонков совпали в апреле 2024.  
3. Конверсия за год снизилась почти в 2 раза (с 0.34 до ~0.18).  
4. Несмотря на рост активности, эффективность обработки лидов падает.

### Возможные причины
- Ухудшение качества лидов (особенно во 2-й половине периода).  
- Перегруз менеджеров звонками — больше лидов, но меньше глубины обработки.  
- Недостаточно эффективные скрипты и последующая работа с клиентами.  

### Рекомендации
- Проанализировать источники лидов: какие каналы дают звонки без конверсии.  
- Усилить обучение менеджеров и оптимизировать скрипты продаж.  
- Добавить метрики качества: средняя длительность звонка, конверсия по источникам.  
- В периоды пиковых нагрузок распределять звонки равномерно (например, автоматизация или доп. сотрудники).


###**2 — Deal Closing Times and Duration**

**What we analyze**

**Closing activity** — как долго длятся сделки от создания до закрытия.

**Duration metrics** — распределение длительности и ключевые статистики.

Method choices — два способа подсчёта длительности:

**Duration_0days**: same-day = 0

**Duration_1day**: same-day = 1 (более понятен)

In [ ]:
# 1. Calculate deal duration (in days) from creation to closing
# Take only closed deals
deals_closed = deals[~deals['_open_deal']].copy()

In [ ]:
# Use only dates (without time) to avoid negative values
deals_closed['created_date'] = pd.to_datetime(deals_closed['Created Time']).dt.floor('D')
deals_closed['closing_date'] = pd.to_datetime(deals_closed['Closing Date']).dt.floor('D')

In [ ]:
# Option A: difference in calendar days (same-day = 0)
deals_closed['Duration_0days'] = (deals_closed['closing_date'] - deals_closed['created_date']).dt.days
deals_closed['Duration_0days']

,Duration_0days
2,0
3,0
4,0
8,0
9,0
...,...
21588,0
21589,56
21590,57
21591,6


- В анализ включены только закрытые сделки (_open_deal == False).

- Открытые сделки выделены флагом и исключены из расчёта длительности.

- Итог по статусу сделок: всего 21 593, из них закрытых 14 645, открытых 6 948.

- Для корректной разницы дат время у Created Time отброшено до полуночи (работаем календарными днями).

In [ ]:
# Safeguard: if the difference < 0 (CRM errors), set to 0
neg_cnt = (deals_closed['Duration_0days'] < 0).sum()
if neg_cnt:
    print(f" Обнаружено {neg_cnt} строк с отрицательной длительностью, обрезаем до 0")
deals_closed['Duration_0days'] = deals_closed['Duration_0days'].clip(lower=0)

 Обнаружено 44 строк с отрицательной длительностью, обрезаем до 0


In [ ]:
# Option B: inclusive (same-day = 1)
deals_closed['Duration_1day'] = deals_closed['Duration_0days'].apply(lambda x: 1 if x == 0 else x)

In [ ]:
# Check statistics for both options
print("\n=== Duration_0days (same-day = 0) ===")
print(deals_closed['Duration_0days'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]))

print("\n=== Duration_1day (same-day = 1) ===")
print(deals_closed['Duration_1day'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]))


=== Duration_0days (same-day = 0) ===
count    14645.000000
mean        14.966815
std         31.484976
min          0.000000
25%          1.000000
50%          3.000000
75%         12.000000
90%         43.000000
95%         78.800000
max        335.000000
Name: Duration_0days, dtype: float64

=== Duration_1day (same-day = 1) ===
count    14645.000000
mean        15.192967
std         31.380069
min          1.000000
25%          1.000000
50%          3.000000
75%         12.000000
90%         43.000000
95%         78.800000
max        335.000000
Name: Duration_1day, dtype: float64


**Duration metrics (days)**


**Variant A — Duration_0days (same-day = 0)**

Mean ≈ 14 дней

Median = 2 дня

P25 / P75 = 1 / 12 дней

P95 ≈ 79 дней

Max ≈ 335 дней

**Variant B — Duration_1day (same-day = 1)**

Mean ≈ 15 дней

Median = 3 дня

P25 / P75 = 1 / 13 дней

P95 ≈ 79 дней

Max ≈ 335 дней

**Итог** -  распределение резко правостороннее: основная масса сделок закрывается быстро (до 1–2 недель), но есть редкие длительные кейсы, которые тянут среднее вверх.

In [ ]:
# Number of deals by Closing Date (monthly)

palette = {
    "orange": "#F5A623",
    "line":   "#D9D9D9",
    "char":   "#2E2E2E",
    "bg":     "rgba(0,0,0,0)"
}

deals_closed['ClosingMonth'] = deals_closed['Closing Date'].dt.to_period('M')
closing_trend = deals_closed.groupby('ClosingMonth').size().reset_index(name='Deals')
closing_trend['ClosingMonth'] = closing_trend['ClosingMonth'].astype(str)

fig1 = px.bar(
    closing_trend,
    x="ClosingMonth",
    y="Deals",
    title="Number of Deals by Closing Month",
    labels={"ClosingMonth": "Closing Month", "Deals": "Number of Deals"},
    text="Deals")

fig1.update_traces(
    marker_color=palette["orange"],
    marker_line_color=palette["line"],
    marker_line_width=1.2,
    textposition="outside")

fig1.update_layout(
    title=dict(
        x=0.0, font=dict(size=18, color=palette["char"])),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=60, r=40, t=60, b=60),
    xaxis=dict(
        title="Closing Month",
        showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(
        title="Number of Deals",
        showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]))

fig1.show()


### Distribution of Closing Dates
- Сделки закрываются **неравномерно по месяцам**.  
- Пик активности пришёлся на **апрель 2024 года**, где число закрытых сделок достигало максимума.  
- После пика наблюдается спад — к июню 2024 количество закрытых сделок сократилось.  
- Это может указывать на сезонность или перегруз в отдельные периоды.

In [ ]:
# Distribution of deal duration (inclusive variant)
# Histogram of Deal Duration (inclusive variant)
palette = {
    "orange": "#F5A623",
    "gold":   "#FF9800",
    "grey":   "#4A4A4A",
    "line":   "#D9D9D9",
    "char":   "#2E2E2E",
    "bg":     "rgba(0,0,0,0)"
}

fig = px.histogram(
    deals_closed,
    x="Duration_1day",
    nbins=50,
    title="Distribution of Deal Duration (days, inclusive variant)",
    labels={"Duration_1day": "Deal Duration (days)", "count": "Number of Deals"})

fig.update_traces(
    marker_color=palette["orange"],
    marker_line_color=palette["line"],
    marker_line_width=1.0)

median_val = deals_closed["Duration_1day"].median()
mean_val = deals_closed["Duration_1day"].mean()

fig.add_vline(
    x=median_val,
    line_width=2, line_dash="dash", line_color=palette["grey"],
    annotation_text=f"Median = {median_val:.0f}",
    annotation_position="top left",
    annotation_font_color=palette["grey"],
    annotation_font_size=11)

fig.add_vline(
    x=mean_val,
    line_width=3, line_dash="dot", line_color=palette["gold"],
    annotation_text=f"Mean = {mean_val:.1f}",
    annotation_position="bottom left",
    annotation_font_color=palette["char"],
    annotation_font_size=12)

fig.update_layout(
    title=dict(x=0.0, font=dict(size=18, color=palette["char"])),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=60, r=40, t=60, b=60),
    xaxis=dict(
        title="Duration (days)",
        showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(
        title="Number of Deals",
        showgrid=False, linecolor=palette["line"], tickcolor=palette["line"]),
    bargap=0.05)

fig.show()


###  Distribution of Deal Duration
- Для анализа использован показатель `Duration_1day` (same-day сделки считаются как 1 день).  
- **Медиана**: ~2–3 дня → половина сделок закрывается очень быстро.  
- **Среднее**: ~15 дней → значение завышено за счёт редких долгих сделок.    
- **5% самых длинных** длятся более 80 дней, а отдельные сделки растягиваются до 11 месяцев (~335 дней).  
- График показывает ярко выраженный пик в начале распределения (быстрые сделки) и длинный «хвост» долгих кейсов.


In [ ]:
# Heatmap: Deal Creation vs Closing
# Duration in days
palette = {
    "gold":   "#FFDFA6",
    "orange": "#F5A623",
    "dark":   "#7A3E00",
    "line":   "#D9D9D9",
    "char":   "#2E2E2E",
    "bg":     "rgba(0,0,0,0)"
}


orange_scale = [
    [0.0, "#FFFFFF"],
    [0.3, palette["gold"]],
    [0.7, palette["orange"]],
    [1.0, palette["dark"]]
]

fig = px.imshow(
    heatmap_df,
    text_auto=".1f",
    aspect="auto",
    color_continuous_scale=orange_scale,
    zmin=0,
    labels=dict(x="Closing Month", y="Creation Month", color="Avg Duration (days)"),
    title="Heatmap of Deal Duration by Creation and Closing Month")

fig.update_layout(
    title=dict(
        x=0.0, font=dict(size=18, color=palette["char"])
    ),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=80, r=60, t=60, b=60),
    xaxis=dict(
        tickangle=45,
        showgrid=False, zeroline=False,
        linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(
        autorange="reversed",
        showgrid=False, zeroline=False,
        linecolor=palette["line"], tickcolor=palette["line"]),
    coloraxis_colorbar=dict(
        title="Avg Duration (days)",
        outlinecolor=palette["line"],
        tickcolor=palette["char"]))

fig.show()


**Основные наблюдения:**

- Большинство сделок закрываются быстро
Основная масса сделок завершается в срок 3–30 дней. Это видно на диагонали карты: чем ближе месяц создания к месяцу закрытия, тем меньше средняя длительность.

- Есть затянутые процессы
В ячейках с длительностью 90–150 дней заметны сделки, которые «зависали» на несколько месяцев (особенно для сделок, созданных осенью 2023 и закрытых весной 2024).

- Аномальные хвосты
Отдельные сделки тянутся 200–335 дней. Они единичные, но сильно искажают картину. Возможные причины:

-- ошибка при закрытии в CRM (сделка оставалась открытой слишком долго);

-- «замороженные» сделки, возобновлённые позже;

-- дубликаты или некорректные данные.

- Положительная динамика
Для сделок, созданных в 2024 году, средняя длительность заметно снижается (часто 2–12 дней). Это может указывать на улучшение процессов работы с клиентами.

### Key Insights
1. Большинство сделок закрываются оперативно (в течение первой недели).  
2. Среднее сильно искажено выбросами, поэтому **для бизнес-анализа лучше ориентироваться на медиану**.  
3. Длинный хвост сделок указывает либо на **особо сложных клиентов**, либо на **ошибки в заполнении CRM**.  
4. Пики по месяцам показывают, когда команда работала с максимальной нагрузкой (апрель 2024) → это может влиять на рост средней длительности.


- Method sensitivity: выбор подсчёта same-day влияет на медиану (+1 день), но картина не меняется — быстрые сделки доминируют.

### Recommendations
- **Мониторинг SLA**: внедрить контроль по длительным сделкам (например, >30 дней).    
- **Управление нагрузкой**: пики апреля могут означать нехватку ресурсов — стоит распределять поток клиентов равномернее.  
- **Data quality**: проверить самые длинные сделки (>200 дней), чтобы убедиться, что это не ошибки в датах.
